In [1]:
from scipy import stats
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.display import Image

# STATISTICAL INFERENCE 1: Regression

the process of deducing properties of an underlying distribution

# Least squares 

**TODO**: generate data from function y(x)=(x-5)^2 + 10, adding Gaussian noise 

In [ ]:
plt.scatter(x,y)
plt.show()

If underlying PDF of data $y$ is i.i.d. as Gaussian, likelihood is: 
$$
{\cal L}(\bf w) = \prod_{i=1}^N  {\cal N}(y_i|\bar{y}_i,\sigma_y)
$$

where 
$$  \bar y_i = \bar y_i({\bf w}) $$

and ${\bf w}$ are the parameters of the machine-learning model.

For a given model, choose the parameters that **maximise** the likelihood (MLE). For Gaussian, equivalent to minimise:
$$
{\rm Cost} = \frac{1}{N}\sum_{i=1}^N \big( y_i - \bar{y}_i({\bf w})\big)^2~~~~~ (\rm MSE)
$$

**TODO**: define Cost function

**Machine Learning procedure:**  choose the model for $\bar y(x)$ that best fit **new data**

Assume a basis of functions like 
$$ f(x) = \beta_0 + \sum_{p=1}^P \beta_p x^p = \beta^T\cdot \vec\phi(x)  $$ 

then the result of minimisation can be computed analytically, and is:
$$
\beta_{\rm ML} = (\Phi^T\Phi)^{-1}\Phi^T\cdot {\bf y}~~~~~~~~~~(1)
$$

In [307]:
def Phi(x,P):
    # x: input vector
    # P: order of polynomial
    xt=x.reshape((-1,1))
    Phi=np.ones(len(x)).reshape((-1,1))
    for p in range(1,P+1):
        Phi=np.hstack((Phi,xt**p))
    return Phi


**TODO**: define Beta function

Idea is to have optimal prediction in data never seen before:
Dataset _randomly_ partitioned into **training** and **test** sets 

**TODO**: reshuffle and partition data in training and test

Let's build the model

In [255]:
def y_pred(Phi,beta):
    # beta: vector of coefficients
    # Phi:  Nx(P+1) matrix of features (inputs)
    return Phi.dot(beta)

For a given model complexity $P$, the training fit is already given by expr.(1) 
evaluated in the training set  (in more complex models, numerical training is needed). The solution for $P=3$ is:

**TODO**: plot results for order 6 polynomial 

**TODO**:Let's evaluate the Cost in the training and in the test sets